In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from keras.layers import Dense,Dropout,Input,Activation,Dropout,BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from keras.models import model_from_json

In [ ]:
train_data = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
for column_name in train_data.columns:
  if (sum(train_data[column_name].isna()) > 0):
    train_data[column_name].fillna(train_data[column_name].mean(), inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data = train_data[train_data['weight']!=0]
y_train = (train_data['resp'] > 0).astype(int)
train_data = train_data.drop(columns = ['ts_id','date','resp','resp_1','resp_2','resp_3','resp_4'])
x_train = np.array(train_data)

In [ ]:
positives = sum(y_train)
negatives = sum(y_train==0)

plt.bar(["Trade", "Dont Trade"], [positives, negatives])

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)

In [ ]:
train_data.head()

In [ ]:
def jsmp_model(input_shape=(131,)):

  X_input = Input(shape = input_shape)

  X = Dense(512)(X_input)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Dense(512)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Dense(512)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Dense(512)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Dense(512)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Dense(1)(X)
  X = Activation('sigmoid')(X)

  model = Model(inputs = X_input, outputs=X)

  return model


In [ ]:
jsmp_model = jsmp_model()
jsmp_model.summary()

In [ ]:
opt = Adam(learning_rate=1e-3)
jsmp_model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])
history = jsmp_model.fit(x_train, np.array(y_train),batch_size = 16384,epochs=500)

In [ ]:
import janestreet
janestreet.make_env.__called__ = False
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
def Score(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.zeros(count_i)
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
predictions = []
resp = []
weight = []
date = []
for (test_df, sample_prediction_df) in iter_test:
    
    
    test_data = test_df

    
    if(np.array(test_data['weight']) > 0):
        test_data = test_df.drop(columns = ['date'])
        predict = jsmp_model.predict(test_data)
        predict = abs(np.ceil(predict - 0.5))
        sample_prediction_df.action = predict
    else:
        sample_prediction_df.action = 0
    
    predictions.append(sample_prediction_df.action)
     #make your 0/1 prediction here
    env.predict(sample_prediction_df)

In [ ]:
index = np.arange(0,len(predictions))


In [ ]:
submission = pd.DataFrame(np.array(predictions),index = index, columns = ['action'])
submission.to_csv("./submission.csv")

In [ ]:
#model_json = jsmp_model.to_json()
#with open("model.json", "w") as json_file:
 #   json_file.write(model_json)
# serialize weights to HDF5
#jsmp_model.save_weights("model.h5")
#print("Saved model to disk")